### Instalando pacotes

In [1]:

!pip install openpyxl
!pip install pandas
!pip install numpy


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import openpyxl

# Avaliação 03
Grupo 5:
*   Allane
*   Item da lista



### Instanciando os grafos
Estamos usando DataFrames da biblioteca pandas para representar as matrizes de adjacencia dos grafos.

In [3]:
matriz_problema_km = pd.read_excel("./PCV__Matriz_do_problema.xlsx", sheet_name="Km")
matriz_problema_min = pd.read_excel("./PCV__Matriz_do_problema.xlsx", sheet_name="Min")
cidades = pd.read_excel("./PCV__Matriz_do_problema.xlsx", sheet_name="Cidades")


### Estrutura da Busca Local

In [4]:
# Funca para obter o custo total a partir de uma solucao (caminho)
def calcular_custo(solucao, grafo):
    if len(solucao) <= 1:
        return
    custo = 0
    for prox in range(1, len(solucao)):
        v1 = solucao[prox - 1]
        v2 = solucao[prox]
        custo += grafo[v1][v2 - 1]
    return custo

# O argumento 'heuristica' eh o algoritmo da busca local que sera realizada (shift, swap etc).
# ele deve retornar uma vizinhanca - a lista de solucoes geradas
def busca_local_primeira_melhoria(solucao, custo_original, heuristica, grafo):
    custo_atual = custo_original
    solucao_atual = solucao
    melhoria = True
    while melhoria:
        melhoria = False
        for nova_solucao in heuristica(solucao_atual):
            novo_custo = calcular_custo(nova_solucao, grafo)
            if novo_custo < custo_atual:
                solucao_atual = nova_solucao
                custo_atual = novo_custo
                melhoria = True
                break
    return solucao_atual, custo_atual


### Heurísticas

In [5]:
def shift(solucao : list):
    solucao_cp = solucao[:]
    for idx_vert_shift in range(1, len(solucao) - 1):
        for idx_destino_shift in range(2, len(solucao) - 1):
            vert_shift = solucao_cp.pop(idx_vert_shift)
            solucao_cp.insert(idx_destino_shift, vert_shift)
            yield solucao_cp

def swap(solucao : list):
    solucao_cp = solucao[:]
    for idx_vert_swap in range(1, len(solucao) - 1):
        for idx_destino_swap in range(2, len(solucao) - 1):
            vert_swap = solucao_cp[idx_vert_swap]
            solucao_cp[idx_vert_swap] = solucao_cp[idx_destino_swap]
            solucao_cp[idx_destino_swap] = vert_swap
            # print("Solucao do swap : ", solucao_cp)
            yield solucao_cp

def inversao(solucao):
    solucao_cp = solucao[:]
    for idx_vert_inicio in range(1, len(solucao) - 1):
        for idx_destino in range(2, len(solucao) - 1):
            sublista_invertida = solucao_cp[idx_vert_inicio:idx_destino]
            sublista_invertida.reverse()
            yield solucao_cp[0:idx_vert_inicio] + sublista_invertida + solucao_cp[idx_destino:]


## Busca por vizinho mais próximo

In [6]:
# Estamos considerando o label numerico
def vizinho_mais_proximo(grafo : pd.DataFrame, vertice_inicial : int | str):
    # Criando conjunto de vertices nao visitados
    # o conjunto contem os indices referentes ao vertice na matriz problema
    vertices_nao_visitados = set(grafo.index)
    percurso = [vertice_inicial]
    vertices_nao_visitados.remove(vertice_inicial - 1)
    dist_total = 0
    v_atual = vertice_inicial
    while len(vertices_nao_visitados) > 0:
        # Obtem o label (numero) do vertice com menor custo
        idx_vizinho_mais_proximo = grafo[v_atual].loc[list(vertices_nao_visitados)].idxmin(skipna=True)
        prox = idx_vizinho_mais_proximo + 1
        dist_total += grafo[v_atual][idx_vizinho_mais_proximo]
        percurso.append(int(prox))
        v_atual = prox
        vertices_nao_visitados.remove(idx_vizinho_mais_proximo)
    dist_total += grafo[v_atual][vertice_inicial -1]
    percurso.append(vertice_inicial)
    return percurso, dist_total


## Aplicando nos problemas

In [7]:
def aplicar_problema(grafo, inicial, heuristica_construtiva, heuristicas_busca_local, out = None):
    caminho, custo = heuristica_construtiva(grafo, inicial)
    melhor_custo = custo
    melhor_caminho = caminho
    heuristica_melhor_solucao = None
    out = True if out is None else out
    if out:
        print("Aplicando Heuristica Construtiva: ", heuristica_construtiva.__name__)
        print(f"Caminho: {caminho}")
        print(f"Custo: {custo}")
        print("Aplicando busca(s) local(is):")
    for h in heuristicas_busca_local:
        solucao_local, custo_local = busca_local_primeira_melhoria(melhor_caminho, melhor_custo, h, grafo)
        if out:
            print(f"Heuristica : {h.__name__}")
            print("Tipo : Primeira Melhoria")
            print(f"Melhor caminho pos busca local: {solucao_local}")
            print(f"Melhor custo pos busca local: {custo_local}")
        if custo_local < melhor_custo:
            melhor_caminho = solucao_local
            melhor_custo = custo_local
            heuristica_melhor_solucao = h.__name__
    return melhor_caminho, melhor_custo, heuristica_melhor_solucao


### Problema 1
Percurso por 48 cidades, partindo de ANGICOS, com funcao custo definida pela distancia em
km.


In [8]:
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km, 1, vizinho_mais_proximo, [swap, shift, inversao])
print(f"Melhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")



Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 48, 40, 25, 17, 7, 19, 16, 34, 33, 20, 38, 37, 47, 39, 35, 42, 26, 32, 31, 36, 13, 24, 29, 43, 14, 46, 5, 41, 6, 27, 30, 44, 28, 18, 15, 45, 1]
Custo: 2291.3999999999996
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 48, 40, 25, 17, 7, 19, 16, 34, 33, 20, 38, 37, 47, 39, 35, 42, 26, 32, 31, 36, 13, 24, 29, 43, 14, 46, 5, 41, 6, 27, 30, 44, 28, 18, 15, 45, 1]
Melhor custo pos busca local: 2291.3999999999996
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 48, 40, 25, 17, 7, 19, 16, 34, 33, 20, 38, 37, 47, 39, 35, 42, 26, 32, 31, 36, 13, 24, 29, 43, 14, 46, 5, 41, 6, 27, 30, 44, 28, 18, 15, 45, 1]
Melhor custo pos busca local: 2291.3999999999996
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos bus


### Problema 2
Percurso por 48 cidades, partindo de ANGICOS, com funcao custo definida pelo tempo em minutos.


In [9]:
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min, 1, vizinho_mais_proximo, [swap, shift, inversao])
print(f"Melhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")


Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 40, 48, 25, 17, 20, 19, 16, 32, 26, 42, 39, 33, 34, 7, 31, 36, 38, 37, 13, 24, 29, 43, 14, 46, 45, 18, 15, 28, 44, 6, 30, 27, 41, 47, 35, 1]
Custo: 2355.0
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 40, 48, 25, 17, 20, 19, 16, 32, 26, 42, 39, 33, 34, 7, 31, 36, 38, 37, 13, 24, 29, 43, 14, 46, 45, 18, 15, 28, 44, 6, 30, 27, 41, 47, 35, 1]
Melhor custo pos busca local: 2355.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 40, 48, 25, 17, 20, 19, 16, 32, 26, 42, 39, 33, 34, 7, 31, 36, 38, 37, 13, 24, 29, 43, 14, 46, 45, 18, 15, 28, 44, 6, 30, 27, 41, 47, 35, 1]
Melhor custo pos busca local: 2355.0
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 23, 2, 3, 4, 41, 30, 6

### Problema 3
Percurso por 36 cidades, partindo de ANGICOS, tomando como função custo a distância
em km

In [10]:
# Filtra as colunas do DataFrame para obter o intervalo de cidades
# [1,36]
matriz_problema_km_36 = matriz_problema_km[list(range(1,37))].iloc[0:36].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_36, 1, vizinho_mais_proximo, [swap, shift, inversao])
print(f"Melhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")


Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 25, 17, 7, 19, 16, 34, 33, 20, 31, 36, 32, 26, 35, 13, 24, 29, 14, 5, 28, 18, 15, 6, 27, 30, 1]
Custo: 1951.4999999999998
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 25, 17, 7, 19, 16, 34, 33, 20, 31, 36, 32, 26, 35, 13, 24, 29, 14, 5, 28, 18, 15, 6, 27, 30, 1]
Melhor custo pos busca local: 1951.4999999999998
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 25, 17, 7, 19, 16, 34, 33, 20, 31, 36, 32, 26, 35, 13, 24, 29, 14, 5, 28, 18, 15, 6, 27, 30, 1]
Melhor custo pos busca local: 1951.4999999999998
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 23, 4, 3, 2, 11, 10, 12, 9, 8, 21, 22, 36, 25, 31, 7, 19, 17, 20, 33, 16, 34, 32, 26, 35, 13, 24, 29, 14, 5, 18, 15, 28, 27, 6, 30

### Problema 4
Percurso por 36 cidades, partindo de ANGICOS, tomando como função custo a o tempo de trajeto
em minutos.


In [11]:
# Filtra as colunas do DataFrame para obter o intervalo de cidades
# [1,36]
matriz_problema_min_36 = matriz_problema_min[list(range(1,37))].iloc[0:36].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_36, 1, vizinho_mais_proximo, [swap, shift, inversao])

print(f"Melhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")



Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 25, 17, 20, 19, 16, 32, 26, 34, 33, 35, 13, 24, 29, 14, 18, 15, 28, 6, 30, 27, 7, 31, 36, 1]
Custo: 1950.0
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 22, 36, 31, 7, 12, 10, 11, 5, 2, 3, 4, 23, 21, 25, 17, 20, 19, 16, 32, 26, 34, 33, 35, 13, 24, 29, 14, 18, 15, 28, 6, 30, 27, 1]
Melhor custo pos busca local: 1864.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 22, 36, 31, 7, 12, 10, 11, 5, 2, 3, 4, 23, 21, 25, 17, 20, 19, 16, 32, 26, 34, 33, 35, 13, 24, 29, 14, 18, 15, 28, 6, 30, 27, 1]
Melhor custo pos busca local: 1864.0
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 23, 4, 3, 2, 5, 11, 10, 12, 9, 8, 21, 22, 36, 25, 31, 7, 17, 20, 33, 19, 34, 16, 32, 26, 35, 13, 24, 29, 14, 18, 15, 28, 30, 6, 27, 1]
Melhor custo pos busca local: 1

### Insercao Mais Barata

In [12]:
import math

# Estamos considerando o label numerico
def insercao_mais_barata(grafo : pd.DataFrame, vertice_inicial : int | str):

    vertices_restantes = set(grafo.index)

    # Selecionamos o indice do vertice mais proximo a raiz de forma gulosa
    v_mais_proximo = grafo[vertice_inicial].idxmin()

    # Ciclo inicial
    rota = [vertice_inicial, v_mais_proximo + 1, vertice_inicial]
    vertices_restantes.remove( vertice_inicial - 1, )
    vertices_restantes.remove(v_mais_proximo)
    vertices_inseridos = {vertice_inicial - 1, v_mais_proximo}

    custo_total = 0
    while vertices_restantes:
        # Estamos selecionando o vertice de 'vertices_restantes' que tem a menor distancia a outro vertice
        # Filtra apenas as linhas da matriz referentes aos vertices do conjunto de vertices_restantes
        r = grafo[list(vertices_restantes)].iloc[list(vertices_inseridos)].min().idxmin()

        # Consegue o indice do vertice do conjunto 'vertices_restantes' que tem a menor distancia
        # ao algum vertice incluido na rota. Para manter este codigo similar ao pseudocodigo visto em sala, chamaremos a variavel de
        # Encontrar a melhor posicao para inserir a rota
        melhor_custo_insercao = math.inf
        melhor_posicao_insercao = None
        for i in range(1, len(rota) - 1):
            u, v = rota[i] - 1, rota[i + 1] - 1
            custo = grafo[u + 1][r] + grafo[r + 1][v] - grafo[u + 1][v]
            if custo < melhor_custo_insercao:
                melhor_custo_insercao = custo
                melhor_posicao_insercao = i

        rota.insert(melhor_posicao_insercao + 1, r + 1)
        vertices_restantes.remove(r)
        vertices_inseridos.add(r)
        custo_total += melhor_custo_insercao
    return rota, calcular_custo(rota, grafo)



### Problema 1
Percurso por 48 cidades, partindo de ANGICOS, com funcao custo definida pela distancia em
km.


In [13]:
melhor_custo, melhor_caminho, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km, 1, insercao_mais_barata, [swap, shift, inversao])

print(f"Melhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")


Aplicando Heuristica Construtiva:  insercao_mais_barata
Caminho: [1, 8, 21, 9, 10, 11, 12, 22, 36, 48, 40, 25, 31, 17, 7, 19, 34, 16, 42, 26, 32, 35, 39, 33, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 45, 44, 41, 23, 1]
Custo: 2148.1
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 23, 8, 21, 9, 10, 11, 12, 22, 36, 48, 40, 25, 31, 17, 7, 19, 34, 16, 42, 26, 32, 35, 39, 33, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 45, 44, 41, 1]
Melhor custo pos busca local: 2148.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 23, 21, 8, 9, 10, 11, 12, 22, 36, 48, 40, 25, 31, 17, 7, 19, 34, 16, 42, 26, 32, 35, 39, 33, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 45, 44, 41, 1]
Melhor custo pos busca local: 2147.2
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 2, 11, 5, 46, 14, 43, 

## Algoritmo Genetico

### Gerador de solucoes aleatorias

In [14]:
import random
def solucao_aleatoria(grafo : pd.DataFrame, vertice_inicial):
    rota = list(map(lambda i : i + 1,grafo.index))
    rota.remove(vertice_inicial)
    random.shuffle(rota)
    return [vertice_inicial] + rota + [vertice_inicial]

# Funca para obter o custo total a partir de uma solucao (caminho)
def calcular_custo(solucao, grafo):
    if len(solucao) <= 1:
        return
    custo = 0
    for prox in range(1, len(solucao)):
        v1 = solucao[prox - 1]
        v2 = solucao[prox]
        # print("v1: ", v1)
        # print("v2: ", v2)
        custo += grafo[v1][v2 - 1]
    return custo

samples = [solucao_aleatoria(matriz_problema_km, 1) for i in range(0,6)]
print(samples)
# custos = [calcular_custo(solucao, matriz_problema_km) for solucao in samples]
# print(f"Minimo : {min(custos)}")
# print(f"Maximo : {max(custos)}")
#

[[1, 31, 43, 33, 35, 8, 14, 6, 27, 41, 23, 9, 46, 15, 4, 39, 30, 47, 17, 5, 24, 13, 48, 45, 40, 42, 2, 7, 32, 16, 22, 34, 18, 36, 10, 28, 19, 26, 44, 21, 37, 25, 11, 38, 3, 20, 12, 29, 1], [1, 8, 20, 32, 24, 6, 42, 11, 17, 13, 23, 36, 5, 26, 27, 9, 30, 35, 45, 47, 38, 19, 14, 15, 39, 4, 28, 41, 31, 46, 48, 33, 3, 25, 2, 10, 34, 16, 40, 44, 18, 12, 29, 43, 21, 37, 7, 22, 1], [1, 47, 21, 24, 25, 31, 27, 10, 40, 39, 26, 33, 38, 34, 5, 37, 29, 45, 19, 23, 13, 41, 8, 22, 12, 9, 18, 2, 11, 4, 15, 46, 14, 20, 35, 16, 3, 43, 32, 17, 36, 28, 42, 6, 44, 48, 30, 7, 1], [1, 45, 35, 2, 37, 3, 9, 41, 26, 25, 39, 16, 40, 33, 23, 12, 22, 17, 24, 6, 46, 44, 20, 43, 28, 36, 14, 42, 10, 5, 4, 13, 19, 34, 48, 27, 11, 21, 8, 38, 7, 30, 18, 15, 29, 47, 31, 32, 1], [1, 20, 18, 36, 46, 40, 3, 29, 43, 5, 42, 24, 38, 30, 6, 47, 34, 16, 45, 10, 25, 12, 11, 32, 41, 35, 33, 2, 9, 48, 28, 22, 17, 15, 23, 13, 27, 39, 7, 8, 44, 4, 21, 26, 37, 14, 19, 31, 1], [1, 11, 48, 9, 24, 18, 43, 41, 39, 37, 12, 35, 40, 45, 10, 

## Gerando populacao
Gerar 15 indivíduos

### Populacao para o problema 1

In [200]:
solucao_insercao_mais_barata = insercao_mais_barata(matriz_problema_km, 1)[0]
solucao_vizinho_mais_proximo = vizinho_mais_proximo(matriz_problema_km, 1)[0]
solucao_insercao_mais_barata_local = aplicar_problema(matriz_problema_km, 1, insercao_mais_barata, [swap, shift, inversao], out=False)[0]
solucao_vizinho_mais_proximo_local = aplicar_problema(matriz_problema_km, 1, vizinho_mais_proximo, [swap, shift, inversao], out=False)[0]

solucoes_aleatorias = [solucao_aleatoria(matriz_problema_km, 1) for i in range(0,36)]

samples = solucoes_aleatorias + [solucao_insercao_mais_barata, solucao_vizinho_mais_proximo, solucao_insercao_mais_barata_local, solucao_vizinho_mais_proximo_local]
# samples = [solucao_insercao_mais_barata, solucao_vizinho_mais_proximo, solucao_insercao_mais_barata_local, solucao_vizinho_mais_proximo_local]

print(samples)
# Ordenando pra facilitar a selecao por elitismo
samples = sorted(samples, key=lambda solucao: calcular_custo(solucao, matriz_problema_km))


[[1, 16, 15, 8, 25, 48, 36, 45, 40, 37, 4, 39, 17, 46, 18, 38, 33, 27, 19, 24, 21, 10, 23, 47, 3, 6, 31, 14, 28, 9, 43, 2, 41, 12, 13, 29, 26, 35, 11, 44, 20, 32, 7, 5, 42, 34, 30, 22, 1], [1, 13, 21, 35, 39, 46, 44, 26, 16, 43, 12, 11, 17, 15, 5, 8, 31, 47, 48, 41, 7, 29, 22, 36, 27, 30, 37, 32, 28, 23, 19, 45, 4, 25, 18, 40, 42, 20, 34, 10, 38, 2, 24, 14, 33, 3, 9, 6, 1], [1, 12, 48, 42, 21, 4, 6, 18, 37, 36, 32, 40, 34, 16, 39, 31, 41, 29, 19, 33, 30, 46, 7, 3, 23, 38, 11, 15, 2, 10, 17, 26, 35, 24, 47, 45, 9, 8, 20, 14, 22, 28, 25, 44, 43, 13, 27, 5, 1], [1, 20, 13, 14, 5, 28, 25, 38, 8, 32, 46, 15, 6, 16, 18, 11, 36, 7, 29, 31, 21, 39, 9, 12, 40, 19, 24, 26, 27, 3, 22, 17, 30, 34, 47, 45, 33, 43, 23, 10, 37, 48, 4, 41, 42, 2, 44, 35, 1], [1, 17, 4, 39, 45, 18, 44, 8, 7, 11, 37, 3, 9, 20, 13, 42, 43, 24, 27, 23, 19, 12, 40, 41, 6, 38, 30, 34, 47, 2, 22, 16, 25, 26, 10, 28, 29, 36, 32, 14, 15, 48, 35, 31, 21, 5, 33, 46, 1], [1, 10, 21, 28, 27, 2, 26, 19, 29, 8, 14, 22, 20, 30, 9, 34

## 1) Inicio
Organizando os cromossomos

In [201]:
cromossomos = [(solucao, calcular_custo(solucao, matriz_problema_km)) for solucao in samples]


## Metodos de Selecao

In [202]:
def elitismo(populacao : list, num_pares):
    populacao_ordenada = sorted(populacao, key=lambda x: x[1])
    genitores = []
    for i in range(0,num_pares):
        genitores.append((populacao_ordenada[i], populacao_ordenada[i + 1]))
    return genitores

def torneio(populacao: list, num_pares: int, tamanho_torneio: int = 3):
  genitores = []
  for _ in range(num_pares):
    competidores_1 = random.sample(populacao, k=tamanho_torneio)
    pai1 = min(competidores_1, key=lambda x: x[1])
    competidores_2 = random.sample(populacao, k=tamanho_torneio)
    pai2 = min(competidores_2, key=lambda x: x[1])
    genitores.append((pai1, pai2))
  return genitores



### Metodos de Cruzamento

In [203]:
def crossover_1_ponto(cromossomo_1, cromossomo_2):
    n = len(cromossomo_1) # Cromossomos tem comprimentos iguais
    # Ponto de corte
    mid = n // 2
    offspring_1 = cromossomo_1[0:mid] + cromossomo_2[mid:n]
    offspring_2 = cromossomo_2[0:mid] + cromossomo_1[mid:n]
    offsprings = [offspring_1, offspring_2]
    # As labels sao numeros em sequencia
    vertices = set(range(2, n))
    for i, offspring in enumerate(offsprings):
        # vertices ja presentes no offspring atual
        vertices_no_offspring = set(offspring)
        vertices_no_offspring.remove(1)
        # se o numero de elementos unicos eh menor que o tamanho - 1 (ja que eh um ciclo),
        # ja elementos repetidos
        if len(vertices_no_offspring) < n-1:
            vertices_restantes = vertices.difference(vertices_no_offspring)
            novo_offspring = offspring[0:mid]
            controle = set(offspring[1:mid])
            for v in offspring[mid:]:
                novo_v = None
                if v in controle:
                    novo_v = vertices_no_offspring.pop()
                else:
                    novo_v = v
                controle.add(novo_v)
                novo_offspring.append(novo_v)
            offsprings[i] = novo_offspring
    return offsprings[0], offsprings[1]

def crossover_ox(genitor_1, genitor_2):
    offspring_1, offspring_2 = genitor_1[:len(genitor_1) // 2], genitor_2[:len(genitor_2) // 2]
    vertices_incluidos1, vertices_incluidos2 = set(offspring_1), set(offspring_2)
    for i in genitor_2:
        if i not in vertices_incluidos1:
            offspring_1.append(i)
    offspring_1.append(genitor_1[0])
    for i in genitor_1:
        if i not in vertices_incluidos2:
            offspring_2.append(i)
    offspring_2.append(genitor_2[0])
    return offspring_1, offspring_2

def crossover_2_pontos(genitor_1, genitor_2):
    offspring_1, offspring_2 = genitor_1[:len(genitor_1) // 2], genitor_2[:len(genitor_2) // 2]
    vertices_incluidos1, vertices_incluidos2 = set(offspring_1), set(offspring_2)
    g1, g2 = random.sample(range(1,len(genitor_1) - 1), 2)
    g_1_inicio, g_1_final = min(g1,g2), max(g1,g2)
    g3, g4 = random.sample(range(1,len(genitor_1) - 1), 2)
    g_2_inicio, g_2_final = min(g1,g2), max(g1,g2)
    for i in genitor_2:
        if i not in vertices_incluidos1:
            offspring_1.append(i)
    offspring_1.append(genitor_1[0])
    for i in genitor_1:
        if i not in vertices_incluidos2:
            offspring_2.append(i)
    offspring_2.append(genitor_2[0])
    return offspring_1, offspring_2
    pass

def crossover_uniforme():
    pass

def cruzamento(genitores, grafo):
    offsprings = []
    for g in genitores:
        genitor_1,genitor_2=g
        offspring1, offspring2 = crossover_ox(genitor_1[0], genitor_2[0])
        offsprings.extend([(offspring1, calcular_custo(offspring1, grafo)), (offspring2,calcular_custo(offspring1, grafo))])
    return offsprings

In [213]:
import random
def mutacao_swap(cromossomos, probabilidade, grafo):
    novos_cromossomos = []
    for cromossomo in cromossomos:
        cromossomo = list(cromossomo[0]) # extrai o caminho

        if random.random() < probabilidade:
            gene_1, gene_2 = random.sample(range(1, len(cromossomo) - 1), 2)
            cromossomo[gene_1], cromossomo[gene_2] = cromossomo[gene_2], cromossomo[gene_1]
        # Adiciona o cromossomo (mutado ou não) com seu novo custo
        novos_cromossomos.append((cromossomo, calcular_custo(cromossomo, grafo)))

    return novos_cromossomos

def mutacao_inversao(cromossomos, probabilidade, grafo):
    novos_cromossomos = []
    for cromossomo_original_tuple in cromossomos:
        cromossomo = list(cromossomo_original_tuple[0]) # Trabalhar com a lista da rota

        if random.random() < probabilidade:
            gene_1, gene_2 = random.sample(range(1, len(cromossomo) - 1), 2)
            gene_inicio = min(gene_1, gene_2)
            gene_fim = max(gene_1, gene_2)
            parte_invertida = cromossomo[gene_inicio:gene_fim][::-1]
            novo = cromossomo[:gene_inicio] + parte_invertida + cromossomo[gene_fim:]

            novos_cromossomos.append((novo, calcular_custo(novo,grafo)))
            # print(f"Custo mutado : {calcular_custo(novo, grafo)}")
        else:
            novos_cromossomos.append(cromossomo_original_tuple)
        # Adiciona o cromossomo (mutado ou não) com seu novo custo

    return novos_cromossomos




In [214]:
# for custo, cromossomo in cromossomos.items():
#     mutacao_swap(cromossomo, 0.5)

In [215]:
a = {1:"a"}
b = random.choice(list(a.items()))
b

(1, 'a')

In [216]:
def renovacao(populacao_atual, offspring, num_populacao):
    pop = populacao_atual + offspring
    # offspring_ordenada = sorted(offspring, key=lambda a : a[1])
    # pop_ordenada = sorted(populacao_atual, key=lambda a : a[1])
    pop_ordenada = sorted(pop, key=lambda x: x[1])
    return pop_ordenada[:num_populacao]
    # Usando torneio
    # nova_populacao = []
    # pop = populacao_atual + offspring
    # while len(nova_populacao) < num_populacao:
    #     candidatos = random.choices(pop, k=3)
    #     # offspring_individuo = random.choice(list(offspring))
    #     escolhido = min(candidatos, key=lambda x: x[1])
    #     nova_populacao.append(escolhido)
    # return nova_populacao

def roleta(populacao, offspring, num_populacao):
    populacao = populacao + offspring
    cromossomos = []
    somatorio_custo = sum(map(lambda cromossomo : cromossomo[1], populacao))
    values = [solucao[1]/somatorio_custo for solucao in populacao]
    fitness = []
    for val in values:
        min_val = min(values)
        max_val = max(values)
        normalized_data = (max_val - val) / (max_val - min_val)
        fitness.append(normalized_data)
    print(f"Custos : ", list(map(lambda x : x[1], populacao)))
    print(f"Fitness : ", fitness)
    solucoes = list(map(lambda c : c[1], populacao))
    return random.choices(populacao, weights=fitness, k=num_populacao)



In [217]:

print(float(random.random()))

0.608130343439807


def algoritmo_genetico(numero_iteracoes, taxa_mutacao, mutacoes, selecao,

In [218]:
def algoritmo_genetico(grafo, populacao_inicial, numero_iteracoes, taxa_mutacao, mutacao, selecao, cruzamento, renovacao):
    populacao_atual = populacao_inicial
    for iter in range(0, numero_iteracoes):
        genitores = selecao(populacao_atual, len(populacao_atual) // 2)
        # print(f"Numero de genitores: {len(genitores)}")
        offsprings = cruzamento(genitores, grafo)
        # Ja aplica a taxa de mutacao a populacao
        offsprings = mutacao(offsprings, taxa_mutacao, grafo)
        # print(f"Tamanho do offspring: {len(offsprings)}")
        # print(offsprings)
        populacao_atual = renovacao(populacao_atual, offsprings, len(populacao_atual))
        # print(f"Tamanho da pop: {len(populacao_atual)}")
    return populacao_atual

custos = []
for i in range(10):
    resultado = algoritmo_genetico(matriz_problema_km, cromossomos, 100, 0.4, mutacao_inversao, elitismo, cruzamento, renovacao)
    custos.append(min(resultado, key=lambda x : x[1])[1])
print(custos)

[np.float64(2084.2999999999997), np.float64(2084.2999999999997), np.float64(2084.2999999999997), np.float64(2084.2999999999997), np.float64(2084.2999999999997), np.float64(2084.2999999999997), np.float64(2084.2999999999997), np.float64(2084.2999999999997), np.float64(2084.2999999999997), np.float64(2084.2999999999997)]


## Memetico

In [219]:
import random

def algoritmo_memetico(grafo, populacao_inicial, numero_iteracoes, taxa_mutacao,
                       mutacao, selecao, cruzamento, renovacao,
                       heuristicas_busca_local, probabilidade_busca_local=0.2):
    populacao_atual = populacao_inicial

    for iteracao in range(numero_iteracoes):

        n_genitores = len(populacao_atual) // 2
        genitores = selecao(populacao_atual, n_genitores)
        print(f"Numero de genitores: {len(genitores)}")
        offsprings = cruzamento(genitores, grafo)
        offsprings = mutacao(offsprings, taxa_mutacao, grafo)

        novos_offsprings = []

        for item in offsprings:
            if isinstance(item, tuple):
                individuo = item[0]
            else:
                individuo = item

            custo_individuo = calcular_custo(individuo, grafo)

            if random.random() < probabilidade_busca_local:
                melhorou = False
                for heuristica in heuristicas_busca_local:
                    nova_rota, novo_custo = busca_local_primeira_melhoria(
                        individuo, custo_individuo, heuristica, grafo
                    )

                    if novo_custo < custo_individuo:
                        individuo = nova_rota
                        custo_individuo = novo_custo
                        melhorou = True

            novos_offsprings.append((individuo, custo_individuo))

        print(f"Tamanho do offspring: {len(novos_offsprings)}")
        print(novos_offsprings)
        print(f"Tamanho da pop: {len(populacao_atual)}")

        populacao_atual = renovacao(populacao_atual, novos_offsprings, len(populacao_atual))

        melhor_custo_geracao = populacao_atual[0][1]
        print(f"Geração {iteracao + 1}/{numero_iteracoes} | Melhor Custo: {melhor_custo_geracao:.2f} | Pop: {len(populacao_atual)}")

    return populacao_atual

In [ ]:
resultado = algoritmo_memetico(
    grafo=matriz_problema_km,
    populacao_inicial=cromossomos,
    numero_iteracoes=20,
    taxa_mutacao=0.3,
    mutacao=mutacao_inversao,
    selecao=elitismo,
    cruzamento=cruzamento,
    renovacao=renovacao,
    heuristicas_busca_local=[swap, shift, inversao]
)

print(resultado)

Numero de genitores: 20
